In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import subprocess
import rpy2
from rpy2 import *
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
#from rpy2.robjects import globalenv
from rpy2.robjects.numpy2ri import numpy2ri
from rpy2.robjects.packages import STAP
#numpy2ri.activate()
from rpy2.robjects import pandas2ri
import shutil
from shutil import copyfile
import math
import re
import fileinput
import tkinter
from tkinter import *
#import tkFont
#from PIL import Image
#from PIL import ImageTk
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di 

di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


#generating 621D features

def feature_extraction(file_name,svalue):
    
    fasta_file_name=file_name + ".fasta"
    lines = [line.rstrip('\n') for line in open(fasta_file_name)]    
    save_sequences= ''
    sequences= []
    sequences_name= []
    for l in lines:
        if(l == ""): #blank lines are disregarded
                pass
        elif (l[0] == '>'):
            sequences_name.append(l[1:])
            sequences.append(save_sequences)
            save_sequences= ''
        else:
            save_sequences+= l

    sequences.append(save_sequences)
    del sequences[0]
    
    
    all_generated_features=pd.DataFrame()
    

     
    #amino acid composition
    
    def aa_composition(seq):
        
        r=ro.r
        r.source("AAC.R") #calling R script to compute amino acid composition
  
        aa_comp = r.extractAAC(seq)
        return aa_comp
    
    
    #composition for CTD model 
    
    def ctd_composition(seq):
        
        r=ro.r
        r.source("CTDC.R") #calling R script to compute composition for CTD model
  
        ctd_comp = r.extractCTDC(seq)
        return ctd_comp
    
    
    #transition for CTD model
    
    def ctd_transition(seq):
        
        r=ro.r
        r.source("CTDT.R") #calling R script to compute transition for CTD model
  
        ctd_trans = r.extractCTDT(seq)
        return ctd_trans
    
    
    #distribution for CTD model
    
    def ctd_distribution(seq):
        
        r=ro.r
        r.source("CTDD.R") #calling R script to compute distribution for CTD model
  
        ctd_distr = r.extractCTDD(seq)
        return ctd_distr
    
    
    #read PSSM matrix
    
    def readToMatrix(input_matrix):
        #print(input_matrix)
        #print "start to read PSSM matrix"
        PSSM = []
        p = re.compile(r'-*[0-9]+')
        for line, strin in enumerate(input_matrix):
            if line > 2:
                str_vec = []
                overall_vec = strin.split()
                #print len(overall_vec)
                if len(overall_vec) == 0:
                    break
                str_vec.extend(overall_vec[1])
                if(len(overall_vec) < 44):
                    print ("There is a mistake in the pssm file")
                    print ("Try to correct it")
                    for cur_str in overall_vec[2:]:
                        str_vec.extend(p.findall(cur_str))
                        if(len(str_vec) >= 21):
                            if(len(str_vec)) >21:
                                exit(1)
                            #print len(str_vec)
                            #print str_vec
                            #print overall_vec
                            #print "Exit with an error"
                            break;
                    print ("Done")
                else:
                    str_vec = strin.split()[1:42]
                if len(str_vec) == 0:
                    break
            #str_vec_positive=map(int, str_vec[1:])
                PSSM.append(str_vec)
        fileinput.close()
        #print "finish to read PSSM matrix"
        PSSM = np.array(PSSM)
        #print(PSSM)
        return PSSM

    #generate 400D matrix
    def preHandleColumns(PSSM,STEP,PART,ID):
    
        if PART==0:
           #print "PART=",PART
           PSSM=PSSM[:,1:21]
        elif PART==1:
             #print "PART=",PART
            PSSM=PSSM[:, 21:]
        PSSM=PSSM.astype(float)
        #print(PSSM)
        matrix_final = [ [0.0] * 20 ] * 20
        matrix_final=np.array(matrix_final)
        seq_cn=np.shape(PSSM)[0]
    #print "seq_cn=",seq_cn

        if ID==0:
            #print "ID=",ID
            for i in range(20):
                for j in range(20):
                    for k in range(seq_cn - STEP):
                        matrix_final[i][j]+=(PSSM[k][i]*PSSM[k+STEP][j])

        elif ID==1:
            #print "ID=",ID
            for i in range(20):
                for j in range(20):
                    for k in range(seq_cn - STEP):
                        matrix_final[i][j] += ((PSSM[k][i]-PSSM[k+STEP][j]) * (PSSM[k][i]-PSSM[k+STEP][j])/4.0)
        #print (matrix_final)
        return matrix_final

    
    #averaging the matrix
    
    def average(matrixSum, seqLen):
        # average the summary of rows
        matrix_array = np.array(matrixSum)
        matrix_array = np.divide(matrix_array, seqLen)
        matrix_array_shp = np.shape(matrix_array)
        matrix_average = [(np.reshape(matrix_array, (matrix_array_shp[0] * matrix_array_shp[1], )))]
        return matrix_average
    
    
    #compute PSSM bigrams
    
    def bigrams_pssm(input_matrix,STEP):
        #print ("start bigrams_pssm function")
        PART=1
        ID=0
        KEY=0
        matrix_final = preHandleColumns(input_matrix, STEP, PART, ID)
        seq_cn = float(np.shape(input_matrix)[0])
        bigrams_pssm_vector=average(matrix_final,10000.0)
        #print ("end bigrams_pssm function")
        #print(bigrams_pssm_vector)
        return bigrams_pssm_vector

    
    #generate secondary structure features
        
    def secondstruct_feat(ssfile):
        
        
        base = importr('base')
        
        r=ro.r
        r.source("GL_updated.R") #calling R script to compute secondary structure features
  
        ss_ft = r.extractSSF(ssfile)
        #print(ssfile)
        return ss_ft
    
    
    #####################################################################################
    
    
    aa_dict='ACDEFGHILKMNPQRSTVWY'
    
    for i in aa_dict:
        all_generated_features["%s"%i]=""
    
    for i in range(1,25):
        all_generated_features["C%s"%i]=""
    
    for i in range(1,25):
        all_generated_features["T%s"%i]=""
        
    for i in range(1,121):
        all_generated_features["D%s"%i]=""
        
    for i in range(1,401):
        all_generated_features["P%s"%i]=""
    
    for i in range(1,34):
        all_generated_features["S%s"%i]=""
    
    
    # Execute for each sequence
    
    seqs_length=[]
    counter=0
    counting_seq=0

    for seq in sequences:
        seqs_length.append(len(seq))
        counter+=1
        aa_comp=aa_composition(seq)
        ctd_comp = ctd_composition(seq)
        ctd_trans = ctd_transition(seq)
        ctd_distr = ctd_distribution(seq)
        
        all_generated_features.at[counting_seq,"A":"Y"]=aa_comp
        all_generated_features.at[counting_seq,"C1":"C24"]=ctd_comp
        all_generated_features.at[counting_seq,"T1":"T24"]=ctd_trans
        all_generated_features.at[counting_seq,"D1":"D120"]=ctd_distr
        
        x_len=len(seq)
        
        #run psi-blast to get pssm file
        myfile = open("pssm_sequence.fasta", "w")
        myfile.write(seq)
        myfile.close()
        #subprocess.call('.\\blast\\bin\\blastpgp -i .\\pssm_sequence.fasta -d uniprot -j 3 -h 0.001 -a 8 -Q .\\pssm_output.pssm', shell=True)
        subprocess.call('./blast/bin/blastpgp_universal-macosx -i ./pssm_sequence.fasta -d uniprot -j 3 -h 0.001 -a 8 -Q ./pssm_output.pssm', shell=True)
        
        pssmContentMatrix=readToMatrix(fileinput.input('pssm_output.pssm'))
        
        #generate pssm bigrams
        pssm_comp=bigrams_pssm(pssmContentMatrix, 1)
        pssm_comp=np.array(pssm_comp)
    
        pssm_comp = np.resize(pssm_comp, (1,400))
        all_generated_features.at[counting_seq,"P1":"P400"]=pssm_comp
        
        #run PSIPRED to get secondary structure probability matrix
        copyfile("pssm_sequence.fasta", "psitmp.fasta")
        #subprocess.call('.\\blast\\bin\\blastpgp -b 0 -j 3 -h 0.001 -d uniprot -i psitmp.fasta -C psitmp.chk -Q out.ckp > temp.blast', shell = True)
        subprocess.call('./blast/bin/blastpgp_universal-macosx -b 0 -j 3 -h 0.001 -d uniprot -i psitmp.fasta -C psitmp.chk -Q out.ckp > temp.blast', shell = True)
        subprocess.call('echo psitmp.chk > psitmp.pn', shell = True)
        subprocess.call('echo psitmp.fasta > psitmp.sn', shell = True)
        #subprocess.call('.\\blast\\bin\\makemat -P psitmp',shell=True)
        subprocess.call('./blast/bin/makemat -P psitmp',shell=True)
        #subprocess.call('seq2mtx pssm_sequence.fasta >pssm_sequence.mtx', shell=True)
        subprocess.call ('psipred psitmp.mtx weights.dat weights.dat2 weights.dat3 > out.ss', shell = True)
        
        #get path for current directiry
        dir_path = os.getcwd()
        dir_path = dir_path.replace("\\", "/")
        dir_path+="/out.ss"

        ss_feat = secondstruct_feat(dir_path)
        
        all_generated_features.at[counting_seq,"S1":"S33"]=ss_feat
        
        counting_seq+=1
    
    if svalue==0:
        all_generated_features=all_generated_features.drop(['A','C','D','E','F','G','H','I','L','K','M','N','P','Q','R','S','T','V','W','Y'], axis=1) ###remove for aa
    elif svalue==1:
        all_generated_features=all_generated_features.drop(['A','C','D','F','G','H','L','K','M','N','P','Q','R','V','W','Y'], axis=1) ###remove for aa
    elif svalue==2:
        all_generated_features=all_generated_features.drop(['A','C','D','E','F','G','H','I','L','K','M','N','P','Q','R','S','T','V','W','Y'], axis=1) ###remove for aa
    elif svalue==3:
        all_generated_features=all_generated_features.drop(['A','C','D','E','F','G','H','I','L','K','M','N','P','Q','R','S','T','V','W','Y'], axis=1) ###remove for aa
    elif svalue==4:
        all_generated_features=all_generated_features.drop(['A','C','D','E','F','G','H','I','K','M','N','P','Q','R','S','T','V','W'], axis=1) ###remove for aa    
    
    eliminate= []
    
    if svalue==0:
        features_selected=["S5", "D80", "C21", "S4", "T10", "D81"]
    elif svalue==1:
        features_selected=["S12","S","T","T15","S29","D37","D72","S14","S15","D16", "D94", "D87","D10","S21","E","D81","C2","I"]
    elif svalue==2:
        features_selected=["S5","D86","D85","D81","C15"]
    elif svalue==3:
        features_selected=["C19","D119","D63","D105","D74","D88","D97","D44","D107","D73","D91","D109","D45","D99","D56","T9"]
    elif svalue==4:
        features_selected=["L","D12","D11","Y","D9","T18","P85","D111","T19","D114","S33","P89","P98","P345","D102","D14","D44","D58"]

    for i in range(1,25):
        a="C%s"%i
        if a not in features_selected:
            eliminate.append(a)
    
    for i in range(1,25):
        a="T%s"%i
        if a not in features_selected:
            eliminate.append(a)
    
    for i in range(1,121):
        a="D%s"%i
        if a not in features_selected:
            eliminate.append(a)  
    
    for i in range(1,401):
        a="P%s"%i
        if a not in features_selected:
            eliminate.append(a)
            
    for i in range(1,34):
        a="S%s"%i
        if a not in features_selected:
            eliminate.append(a)
    
    all_generated_features=all_generated_features.drop(eliminate, axis=1)
    
    if svalue==0:
        all_generated_features=all_generated_features[['S5', 'D80', 'C21', 'S4', 'T10', 'D81']]
    elif svalue==1:
        all_generated_features=all_generated_features[['S12','S','T','T15','S29','D37','D72','S14','S15','D16', 'D94', 'D87','D10','S21','E','D81','C2','I']]
    elif svalue==2:
        all_generated_features=all_generated_features[['S5','D86','D85','D81','C15']]
    elif svalue==3:
        all_generated_features=all_generated_features[['C19','D119','D63','D105','D74','D88','D97','D44','D107','D73','D91','D109','D45','D99','D56','T9']]
    elif svalue==4:
        all_generated_features=all_generated_features[['L','D12','D11','Y','D9','T18','P85','D111','T19','D114','S33','P89','P98','P345','D102','D14','D44','D58']]

    all_generated_features.to_csv("%s.csv" %file_name, header=True, index=False)
    



#predicting antibiotic resistance sequences
def predict_resistance_sequences(svalue):
    
    #reading training and test datsets 
    
    dir_path = os.getcwd()
    dir_path = dir_path.replace("\\", "/")
    if svalue==0:
        training_file_path=dir_path + "/training_set_aac.csv" #path for aac training set
    elif svalue==1:
        training_file_path=dir_path + "/training_set_bla.csv" #path for bla training set
    elif svalue==2:
        training_file_path=dir_path + "/training_set_dfr.csv" #path for dfr training set     
    elif svalue==3:
        training_file_path=dir_path + "/training_set_bac.csv" #path for bac training set     
    elif svalue==4:
        training_file_path=dir_path + "/training_set_van.csv" #path for van training set
    
    testing_file_path=dir_path + "/input_seq.csv" #path for input sequences
    #print(training_file_path)
    #print(testing_file_path)
    #base = importr('base')
    #utils = rpackages.importr('utils')
    #utils.chooseCRANmirror(ind=1) # select the first mirror in the list
    #packnames = ('ROSE', 'e1071', 'caret') 
    #utils.install_packages(StrVector(packnames))
    r=ro.r
    r.source("file_with_pkgTest.R") #check whether required R package is installed
    #r.pkgTest("ROSE")
    r.pkgTest("base")
    r.pkgTest("e1071") #install e1071 R package if not installed
    #r.pkgTest("caret")
    #r.source("SVM_prediction.R") #calling R script to get prediction results from SVM
    r.source("SVM_classifier_revised.R") #calling R script to get prediction results from SVM
    predictions = r.predict_results(training_file_path,testing_file_path)
    #print(predictions)
    #print(predictions[1])
    
    
    # #################################################################
    lines = [line.rstrip('\n') for line in open('input_seq.fasta')]
    

    #reading all sequences in sequences and their names in sequences_name
    save_sequences= ''
    sequences_input= []
    sequences_name_input= []


    for l in lines:
        if(l == ""): #blank lines are disregarded
                pass
        elif (l[0] == '>'):
            sequences_name_input.append(l[1:])
            sequences_input.append(save_sequences)
            save_sequences= ''
        else:
            save_sequences+= l

    sequences_input.append(save_sequences)
    del sequences_input[0]
    
 
  #generate prediction statistics

    predict_file = open("predicted_resistance_sequences.fasta", "w+")

    count_resistance_sequences=0   
    for i in range(len(predictions)):
        if (predictions[i]==1 ):
            predict_file.write(str(sequences_name_input[i])+ '\n'+ '\n')
            count_resistance_sequences+=1
    predict_file.close()
    del predict_file


    one_line="Total number of predicted resistance sequences = "+str(count_resistance_sequences) + "\n"  +"\n" 
    with open("predicted_resistance_sequences.fasta", 'r+') as fp:
        lines = fp.readlines()     
        lines.insert(0, one_line)  
        fp.seek(0)                 
        fp.writelines(lines)  


#include new sequences to the training and test datasets 

def add_new_sequences(svalue,sgval): 
    file_new = open("input_seq.fasta", "r")
    data_new = file_new.read()
    file_new.close()
    
    if svalue==0:
        file_all= open("training_set_aac.fasta","a")
    elif svalue==1:
        file_all= open("training_set_bla.fasta","a")
    elif svalue==2:
        file_all= open("training_set_dfr.fasta","a")      
    elif svalue==3:
        file_all= open("training_set_bac.fasta","a")      
    elif svalue==4:
        file_all= open("training_set_van.fasta","a")
    
    file_all.write("\n")
    file_all.write(data_new)
    file_all.write("\n")
    file_all.close()
        
    feature_extraction('input_seq',svalue)
    df1 = pd.read_csv('input_seq.csv')
    if sgval==1:
        Otpt = [1] * df1.shape[0]
    else:
        Otpt = [-1] * df1.shape[0]
    df1["Output"]= Otpt
    df1.to_csv("seq_excld_header.csv", header=False, index=False)
    file_new_features = open("seq_excld_header.csv", "r")
    data_new_features = file_new_features.read()
    file_new_features.close()
    
    if svalue==0:
        file_all_features= open("training_set_aac.csv","a")
    elif svalue==1:
        file_all_features= open("training_set_bla.csv","a")
    elif svalue==2:
        file_all_features= open("training_set_dfr.csv","a")      
    elif svalue==3:
        file_all_features= open("training_set_bac.csv","a")      
    elif svalue==4:
        file_all_features= open("training_set_van.csv","a")
    
    file_all_features.write(data_new_features)
    file_all_features.close()

    # reset training and test data sets
def restore_training_data():
    copyfile("training_set_aac_actual.fasta", "training_set_aac.fasta")
    copyfile("training_set_bla_actual.fasta", "training_set_bla.fasta")
    copyfile("training_set_dfr_actual.fasta", "training_set_dfr.fasta")
    copyfile("training_set_bac_actual.fasta", "training_set_bac.fasta")
    copyfile("training_set_van_actual.fasta", "training_set_van.fasta")
    
    copyfile("training_set_aac_actual.csv", "training_set_aac.csv")
    copyfile("training_set_bla_actual.csv", "training_set_bla.csv")
    copyfile("training_set_dfr_actual.csv", "training_set_dfr.csv")
    copyfile("training_set_bac_actual.csv", "training_set_bac.csv")
    copyfile("training_set_van_actual.csv", "training_set_van.csv")
    
    

    
#build graphical user interface

root = tkinter.Tk()
root.title("PARGT")
root.geometry("520x485")
root.configure(background="white smoke")
#root.wm_attributes('-alpha', 0.7)


#canvas = Canvas(root, width=235, height = 139)  
canvas = Canvas(root, width=259, height = 273, bg='white smoke',highlightthickness=0)
#canvas = Canvas(root, width=362, height = 352, bg='white smoke',highlightthickness=0)
canvas.pack(padx=5, pady=5)


#add logo of the tool

#width=389
#height=398
#img = Image.open("AS_logo_design.gif")
img = PhotoImage(file="PARGT_logo.gif") 
#img = img.resize((width,height), Image.ANTIALIAS)
#photoImg =  ImageTk.PhotoImage(img)

#canvas.create_image(120,71, anchor=CENTER, image=img) 
canvas.create_image(140,145, anchor=CENTER, image=img)
#canvas.create_image(170,130, anchor=CENTER, image=photoImg)


#T = Text(root,bg="olive", fg="black", font="none 12 bold",bd=2,height=2, width=33, padx=10, pady=10)



#T.pack(padx=10, pady=10)
#T.insert(END, "         Predicting Antibiotic Resistance\n                    via Game Theory","center")


#setup window

def setup_window(soption):
    window = Toplevel(root)
    window.geometry("320x60")
    window.configure(background="tan")
    if soption not in choices:
        Lbl = Label(window, bg="tan",fg="black",text="Please select an option")
        Lbl.config(font=('Helvetica', 8, 'bold'))
        Lbl.pack( )
    else:
        Lbl = Label(window, bg="tan",fg="black",text="Operation is successful!")
        Lbl.config(font=('Helvetica', 8, 'bold'))
        Lbl.pack( )
    
    Btn=Button(window, text="OK", command=window.destroy) 
    Btn.config(font=('Helvetica', 8, 'bold'))
    Btn.pack()

    
#peform operation based on choice    
def submit():
    sf = "%s" % var.get()
    if var.get()==choices[0]:
        op_value=0
        feature_extraction('input_seq',op_value)
        predict_resistance_sequences(op_value)
        
    
    elif var.get()==choices[1]:
        op_value=1
        feature_extraction('input_seq',op_value)
        predict_resistance_sequences(op_value)
        
    
    elif var.get()==choices[2]:
        op_value=2
        feature_extraction('input_seq',op_value)
        predict_resistance_sequences(op_value)
        
    
    elif var.get()==choices[3]:
        op_value=3
        feature_extraction('input_seq',op_value)
        predict_resistance_sequences(op_value)
        
    
    elif var.get()==choices[4]:
        op_value=4
        feature_extraction('input_seq',op_value)
        predict_resistance_sequences(op_value)
        
    
    elif var.get()==choices[5]:
        op_value=0
        signvalue=1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[6]:
        op_value=1
        signvalue=1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[7]:
        op_value=2
        signvalue=1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[8]:
        op_value=3
        signvalue=1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[9]:
        op_value=4
        signvalue=1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[10]:
        op_value=0
        signvalue=-1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[11]:
        op_value=1
        signvalue=-1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[12]:
        op_value=2
        signvalue=-1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[13]:
        op_value=3
        signvalue=-1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[14]:
        op_value=4
        signvalue=-1
        add_new_sequences(op_value,signvalue)
    
    elif var.get()==choices[15]:
        restore_training_data()
    
    
    setup_window(sf)


var = tkinter.StringVar(root)
# initial value
var.set('< Select option (aac/bla/dfr for Gram-negative, and bac/van for Gram-positive) >')
choices = ['Predict aac resistance sequences', 'Predict bla resistance sequences', \
           'Predict dfr resistance sequences', 'Predict bac resistance sequences', \
           'Predict van resistance sequences', 'Include new aac resistance sequences', \
           'Include new bla resistance sequences', 'Include new dfr resistance sequences', \
           'Include new bac resistance sequences', 'Include new van resistance sequences', \
           'Include new aac non-resistance sequences', 'Include new bla non-resistance sequences', \
           'Include new dfr non-resistance sequences', 'Include new bac non-resistance sequences', \
           'Include new van non-resistance sequences', 'Restore training sets']
option = tkinter.OptionMenu(root, var, *choices)
#option.config(bg = "GREEN")
#helv35=font.Font(family='Helvetica', size=36)
option.config(font=('Helvetica', 8, 'bold')) 
#option["menu"].config(bg="GREEN")
option["menu"].config(font=('Helvetica', 8, 'bold'))
option.pack( padx=10, pady=40)
button = tkinter.Button(root, text="Submit", command=submit)
button.config(font=('Helvetica', 8, 'bold'))
button.pack(padx=10, pady=10)


root.mainloop()

Toggle code